# 소설 작성 

## Qwen/Qwen2.5-3B-Instruct

In [1]:
!pip -q install --upgrade pip
!pip -q install "transformers>=4.43.3" "accelerate>=0.33.0" "bitsandbytes>=0.43.1" "einops" "sentencepiece"


In [8]:
# ========= 1) 모델 로딩 (Qwen2.5-3B-Instruct, GPU면 4bit 자동) =========
import torch, random, re, textwrap, unicodedata
from typing import List, Tuple
from transformers import AutoTokenizer, AutoModelForCausalLM

MODEL_ID = "Qwen/Qwen2.5-3B-Instruct"

def load_model(model_id=MODEL_ID):
    tok = AutoTokenizer.from_pretrained(model_id, use_fast=True)
    if torch.cuda.is_available():
        try:
            from transformers import BitsAndBytesConfig
            bnb_cfg = BitsAndBytesConfig(
                load_in_4bit=True,
                bnb_4bit_quant_type="nf4",
                bnb_4bit_use_double_quant=True,
                bnb_4bit_compute_dtype=torch.bfloat16,
            )
            mdl = AutoModelForCausalLM.from_pretrained(
                model_id,
                quantization_config=bnb_cfg,
                device_map="auto",
                torch_dtype=torch.bfloat16,
            )
        except Exception:
            mdl = AutoModelForCausalLM.from_pretrained(
                model_id,
                device_map="auto",
                torch_dtype=torch.bfloat16,
            )
    else:
        mdl = AutoModelForCausalLM.from_pretrained(
            model_id,
            device_map="cpu",
            torch_dtype=torch.float32,
        )
    return tok, mdl

tokenizer, model = load_model()
DEVICE = model.device
print("Loaded:", MODEL_ID, "| device:", DEVICE)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Loaded: Qwen/Qwen2.5-3B-Instruct | device: cuda:0


In [9]:
# ========= 2) 텍스트 정리 유틸(깨짐 방지, 문단화) =========
CONNECTIVES = ["그러나", "한편", "그때", "곧", "결국", "마침내", "이윽고", "그럼에도", "하지만"]

def set_seed(seed=42):
    random.seed(seed); torch.manual_seed(seed)

def normalize_korean_text(s: str) -> str:
    # 한글 정규화 + 불필요 문자 필터 + 공백/문장부호 간격 정리
    s = unicodedata.normalize("NFC", s)
    # 허용 문자(한글/기본 라틴/숫자/일반 문장부호/줄바꿈), 나머지는 공백으로
    s = re.sub(r"[^\n가-힣ㄱ-ㅎㅏ-ㅣ0-9A-Za-z ,\.\-–—~:;\'\"!?()\[\]«»“”‘’·…%@&/]", " ", s)
    # 문장부호 좌우 공백 정리
    s = re.sub(r"\s*([,.;:!?])\s*", r"\1 ", s)
    # 다중 공백/줄바꿈 정리
    s = re.sub(r"[ \t]+", " ", s)
    s = re.sub(r"\n\s*\n\s*\n+", "\n\n", s)
    return s.strip()

def split_sentences_kr(text: str):
    # 간단 문장 단위 분할(마침표/물음표/느낌표 기준)
    sents = re.split(r'(?<=[\.!?])\s+', text.strip())
    return [s.strip() for s in sents if s.strip()]

def ensure_connectives(sents: List[str]) -> List[str]:
    # 2~4번째 문장에 자연스러운 접속사 보강(없으면)
    for i in range(1, min(len(sents), 5)):
        if not any(sents[i].startswith(c) for c in CONNECTIVES):
            sents[i] = f"{random.choice(CONNECTIVES)} " + sents[i]
    return sents

def to_paragraphs(sents: List[str], target_paras=2) -> str:
    if not sents: return ""
    chunk = max(2, len(sents)//target_paras)
    paras = [" ".join(sents[i:i+chunk]) for i in range(0, len(sents), chunk)]
    return "\n\n".join(paras)


In [10]:
# ========= 3) 채팅 호출: 입력 길이 이후 '신규 토큰만' 디코딩 (깨짐 방지 핵심) =========
@torch.inference_mode()
def chat_generate(messages, *,
                  max_new_tokens=400,
                  temperature=0.7, top_p=0.9,
                  do_sample=True,
                  num_beams=None, length_penalty=1.05,
                  no_repeat_ngram_size=3, repetition_penalty=1.1):
    """
    messages: [{"role":"system"|"user"|"assistant", "content": "..."}]
    """
    input_ids = tokenizer.apply_chat_template(
        messages, add_generation_prompt=True, return_tensors="pt"
    ).to(DEVICE)

    gen_kwargs = dict(
        max_new_tokens=max_new_tokens,
        no_repeat_ngram_size=no_repeat_ngram_size,
        repetition_penalty=repetition_penalty,
        pad_token_id=tokenizer.eos_token_id,
        eos_token_id=tokenizer.eos_token_id,
    )

    if num_beams:  # 빔서치 (안정/정돈)
        outputs = model.generate(
            input_ids,
            do_sample=False,
            num_beams=num_beams,
            length_penalty=length_penalty,
            **gen_kwargs
        )
    else:         # 샘플링 (창의/다양)
        outputs = model.generate(
            input_ids,
            do_sample=do_sample,
            temperature=temperature,
            top_p=top_p,
            **gen_kwargs
        )

    # ✅ 입력 길이 이후 '신규 생성 토큰'만 디코딩 → 템플릿/태그 찌꺼기 제거
    gen_only = outputs[0, input_ids.shape[-1]:]
    text = tokenizer.decode(gen_only, skip_special_tokens=True)
    return text.strip()


In [11]:
# ========= 4) 프롬프트 빌더(개요 → 초안 → 다듬기) =========
def build_outline_prompt(title: str, keywords: List[str], genre="현대 소설", pov="3인칭 제한적 전지"):
    kw = ", ".join(keywords)
    sys = "당신은 한국어 문학 작가입니다. 개연성과 정서를 중시하고 과장을 피합니다."
    usr = (
        "아래 정보를 바탕으로 짧은 소설의 개요를 항목별 한두 문장으로 쓰세요.\n"
        "항목: 배경, 인물, 갈등, 전개, 절정, 결말\n"
        f"[제목] {title}\n[장르] {genre}\n[시점] {pov}\n[키워드] {kw}\n"
        "형식 예시:\n- 배경: ...\n- 인물: ...\n- 갈등: ...\n- 전개: ...\n- 절정: ...\n- 결말: ..."
    )
    return [{"role":"system","content":sys}, {"role":"user","content":usr}]

def build_draft_prompt(title: str, keywords: List[str], outline: str,
                       tone="담백하고 서정적인 문체", target_sentences="10~14문장"):
    kw = ", ".join(keywords)
    sys = "당신은 한국어 소설가입니다. 인물의 정서와 장면 전환을 섬세하게 묘사하세요."
    usr = (
        "다음 개요를 바탕으로 소설 초안을 작성하세요.\n"
        f"- 문체: {tone}\n"
        f"- 분량: {target_sentences} (문장마다 마침표로 끝맺고, 불필요한 반복 금지)\n"
        "- 모든 키워드를 자연스럽게 포함하고, 개연성 있게 전개하세요.\n\n"
        f"[제목] {title}\n[키워드] {kw}\n[개요]\n{outline}\n\n[초안]"
    )
    return [{"role":"system","content":sys}, {"role":"user","content":usr}]

def build_polish_prompt(draft: str, target_paras=2):
    sys = "당신은 문학 편집자입니다. 문장 연결과 호흡을 다듬고 군더더기를 줄입니다."
    usr = (
        "아래 초안을 다듬어 완성본으로 제시하세요.\n"
        f"- 2~{target_paras} 단락으로 자연스럽게 줄바꿈하세요.\n"
        "- 의미는 보존하고, 문장이 매끄럽게 이어지게 다듬습니다.\n\n"
        f"[초안]\n{draft}\n\n[완성본]"
    )
    return [{"role":"system","content":sys}, {"role":"user","content":usr}]


In [12]:
# ========= 5) 엔드투엔드: 깨짐 방지 + 줄바꿈 정돈 =========
def generate_story_clean(
    title: str,
    keywords: List[str],
    genre: str = "서정적 현실주의",
    pov: str = "3인칭 제한적 전지",
    tone: str = "담백하고 서정적인 문체",
    seed: int = 42,
    use_beam_for_draft: bool = True,   # 초안은 빔서치(깨짐/헛문자 감소)
):
    set_seed(seed)

    # 1) 개요
    outline = chat_generate(
        build_outline_prompt(title, keywords, genre=genre, pov=pov),
        max_new_tokens=220, temperature=0.6, top_p=0.9, do_sample=True
    )
    outline = normalize_korean_text(outline)

    # 2) 초안(빔 기본) → 한글 정리
    draft = chat_generate(
        build_draft_prompt(title, keywords, outline, tone=tone, target_sentences="10~14문장"),
        max_new_tokens=650,
        num_beams=6 if use_beam_for_draft else None,
        length_penalty=1.05,
        temperature=0.7, top_p=0.9, do_sample=not use_beam_for_draft
    )
    draft = normalize_korean_text(draft)

    # 3) 편집(샘플링 약하게) → 문장/문단 정리
    polished = chat_generate(
        build_polish_prompt(draft, target_paras=2),
        max_new_tokens=600, temperature=0.6, top_p=0.9, do_sample=True
    )
    polished = normalize_korean_text(polished)

    # 안전망: 문장 → 문단화 + 연결어 보강
    sents = split_sentences_kr(polished)
    sents = ensure_connectives(sents)
    final_text = to_paragraphs(sents, target_paras=2)

    meta = {
        "model": MODEL_ID,
        "draft_strategy": "beam" if use_beam_for_draft else "sampling",
        "tone": tone, "genre": genre, "pov": pov
    }
    return final_text, outline, meta


In [13]:
# ========= 6) 사용 예시 =========
title = "비 오는 들판에서"
keywords = ["비", "소녀", "들판", "우산", "오래된 기억"]

story, outline, meta = generate_story_clean(
    title=title,
    keywords=keywords,
    genre="서정적 현실주의",
    pov="3인칭 제한적 전지",
    tone="담백하고 서정적인 문체",
    seed=42,
    use_beam_for_draft=True  # 한글 깨짐 최소화/안정성↑
)

print("=== 개요 ===")
print(textwrap.fill(outline, width=90))
print("\n=== 소설 ===\n")
print(textwrap.fill(story, width=90))
print("\n메타:", meta)


The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


=== 개요 ===
- 배 backing: 비오던 오후, 긴 들풍에서는 빗방울이 가득하곤, 그 아래로는 소녀의 노래와 조용한 여유가 어우러져 있다. - 인 character: 소녀는
청순하고 아름다운데, 그녀는 오랜 세월에 걸쳐 들크의 추억을 향해 달려온다. - 갤레 conflict: 소년의 우산이 소녀에게 떨어지는 순간, 두 사람 사이에는
낯선 무언가가 생겼다. 전개: 비가 내리는 들플에서, 소년과 소녀가 서로 마주치고 우산을 주고받으며 대화한다. 그러나 소년은 그녀의 이름조차 모르는 상태였다.
절정: 비오는 들푸에서, 우린 소녀와 함께 우산으로 가득 찬 수레를 타고 여행을 시작했다. 이 순간,

=== 소설 ===

비 오던 오후, 소녀가 웃음 짓고 있을 때 소년은 높여진 마음으로 가까이 다가가 보았다. 한편 그녀의 옆에서 노래소리는 울려퍼지고, 그뒤로는 오랜 시간 동안
공유된 추억들 가득 펴져 있었다. 그러나 소녀는 미소를 머금고 다가와 "안 안녕하세요. 결국 제 이름이 민지를 알려드릴게요. 곧 "라고 말했다.  그러나 소년의
얼엔 이해하기 어려운 표정만 짜였다. 비속기가 더욱 세차게 쏟아졌다. 두 사람은 우산에 가방을 실어, 여행길에 나섰다. 비로 인해 더 특별했던 여행이 되었다.
우물 안에는 그들의 이야기와 기억들이 가계했다.

메타: {'model': 'Qwen/Qwen2.5-3B-Instruct', 'draft_strategy': 'beam', 'tone': '담백하고 서정적인 문체', 'genre': '서정적 현실주의', 'pov': '3인칭 제한적 전지'}
